In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 14 21:27:46 2018

@author: 20584059
"""
from __future__ import division
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot as plt
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,GRU
from keras.layers import Dropout
import numpy as np
from keras.optimizers import Adam
from keras import layers
from functools import reduce
#from phased_lstm_keras.PhasedLSTM import PhasedLSTM as PLSTM
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Activation
from numpy import concatenate
import bs4 as bs
import urllib.request
import pandas as pd
from pandas import ExcelWriter  
def scale(a):
	a_min = np.nanmin(a, 0)
	a_max = np.nanmax(a, 0)
	return (a - a_min) / (a_max - a_min), a_min, a_max


def unscale(a_sc, amin, amax):
	return a_sc * (amax - amin) + amin


def factors(n):
	return set(reduce(list.__add__,
					  ([i, n // i] for i in range(1, int(pow(n, 0.5) + 1)) if n % i == 0)))


def NSE(y_observed, y_predicted):
	NSE_coefficient = 1 - sum((y_predicted[-2] - y_observed[:-2]) ** 2) / sum((y_observed - np.mean(y_observed)) ** 2)
	return NSE_coefficient
def MAPE(y_observed, y_predicted):
	MAPE_coefficient = np.mean((abs(y_observed-y_predicted)/y_observed))*100
	return MAPE_coefficient
def RMSE(y_observed, y_predicted):
	RMSE_coefficient = np.sqrt(np.mean((y_predicted - y_observed[:-1]) ** 2))
	return RMSE_coefficient
dataset = read_csv('/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Final_Lag24_storage_Data.csv', header=0,index_col=0)
#dataset = dataset.drop(['Chd_Cereals(t) / PERCENTILE(Nig_Pigss)'],axis = 1)
values = dataset.values
#--- reading the dataset, splitting and defining the input/output data
def data_preparation():
    dataset = read_csv('/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Final_Lag24_storage_Data.csv', header=0,index_col=0)
    #dataset = dataset.drop(['Chd_Cereals(t) / PERCENTILE(Nig_Pigss)'],axis = 1)
    values = dataset.values
    values = values.astype('float32')
    scaled, value_min, value_max = scale(values)
    n_hours_forecast = 1
    n_hours = 1
    n_features = 24
    values = scaled
    n_train_hours = 504
    n_test_hours  = 564
    val_test_hours= 600
    train = values[:n_train_hours, :]
    test  = values[n_train_hours :n_test_hours,:]
    val   = values[n_test_hours: val_test_hours,:]
    # split into input and outputs
    n_obs = n_hours * n_features
    return n_obs,train,test,val,value_min,value_max
n_obs,train,test,val,value_min,value_max=data_preparation()
n_hours_forecast = 1
n_hours = 1
n_features = 24
steps_before = 1
steps_after=1
feature_count = n_features
n_obs = n_hours * n_features
# many(18*4) to many(6*4)
train_X, train_y = train[:, :n_obs], train[:, n_obs:]
print(train_X.shape,train_y.shape)
test_X, test_y = test[:, :n_obs], test[:, n_obs:]
print(test_X.shape,test_y.shape)
val_X, val_y   = val[:, :n_obs], val[:, n_obs:]
print(val_X.shape,val_y.shape)
# need to reshape X arrays for use with LSTM
train_X_reshaped = train_X.reshape((train_X.shape[0], n_hours, n_features))
test_X_reshaped  = test_X.reshape((test_X.shape[0], n_hours, n_features))
val_X_reshaped   = val_X.reshape((val_X.shape[0], n_hours, n_features))
# need to reshape Y arrays for use with LSTM
train_y_reshaped = train_y.reshape((train_y.shape[0], n_hours_forecast, n_hours_forecast))
test_y_reshaped  = test_y.reshape((test_y.shape[0], n_hours_forecast, n_hours_forecast))
val_y_reshaped   = val_y.reshape((val_y.shape[0], n_hours_forecast, n_hours_forecast))
def baseline_model():
    #create model
    model = Sequential()
    model.add(LSTM(20, input_shape=(train_X_reshaped.shape[1], train_X_reshaped.shape[2]), return_sequences=False, activation='tanh'))
    model.add(Dropout(0.2))
    # model.add(LSTM(2, kernel_regularizer=L1L2(0.01, 0.01)))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def features_prediction(n_obs,train,test):
    #trainNSE_calc_list = []
    trainMAPE_calc_list = []
    trainRMSE_calc_list = []
    #testNSE_calc_list = []
    testMAPE_calc_list = []
    testRMSE_calc_list = []
    #valNSE_calc_list = []
    valMAPE_calc_list = []
    valRMSE_calc_list = []
    train_y_unscaled_list = []
    ytrain_predicted_unscaled_list= []
    test_y_unscaled_list = []
    ytest_predicted_unscaled_list = []
    val_y_unscaled_list = []
    yval_predicted_unscaled_list  = []
    trainstd_calc_list = []
    teststd_calc_list  = []
    valstd_calc_list   = []
#    trainResults = DataFrame()
#    testResults  = DataFrame()
#    valResults   = DataFrame()
    for i in range(1):
        train_X, train_y = train[:, :n_obs], train[:, i]
        test_X, test_y = test[:, :n_obs], test[:, i]
        val_X, val_y   = val[:, :n_obs], val[:, i]
        train_X_reshaped = train_X.reshape((train_X.shape[0], n_hours, n_features))
        test_X_reshaped = test_X.reshape((test_X.shape[0], n_hours, n_features))
        val_X_reshaped   = val_X.reshape((val_X.shape[0], n_hours, n_features))
        model = KerasRegressor(build_fn=baseline_model, epochs=20, batch_size=30, verbose=0)
        #model.fit(train_X_reshaped,train_y,verbose=2)
        
        for j in range(30):
            history = model.fit(train_X_reshaped, train_y, epochs=2000, batch_size=30, validation_data=(test_X_reshaped, test_y), verbose=1, shuffle=False)
            pd.DataFrame(model.fit(train_X_reshaped, train_y, epochs=2000, batch_size=30, validation_data=(test_X_reshaped, test_y), verbose=1, shuffle=False).history).to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Loss_history/24Lags/UVSTO_history"+str(j)+".csv")
            plt.plot(history.history['loss'], label='CLUGR train set')
            plt.plot(history.history['val_loss'], label='CLUGR test set')
            plt.legend()
            plt.xlabel('Epochs')
            plt.ylabel('Loss')
            plt.show()
            #Predict on the trainset
            ytrain_predicted = model.predict(train_X_reshaped)
            ytrain_predicted_unscaled = unscale(ytrain_predicted,value_min[0],value_max[0])
            ytrain_predicted_unscaled_list.append(ytrain_predicted_unscaled)
            train_y_unscaled = unscale(train_y, value_min[0], value_max[0])
            train_y_unscaled_list.append(train_y_unscaled)
            forecast32 = ytrain_predicted_unscaled.reshape(-1,1)
            forecast32 = pd.DataFrame(forecast32)
            forecast32.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/TrainsetCSVs/24Lags/trainsetcsv"+str(j)+".csv")
            #Predict on the testset
            ytest_predicted = model.predict(test_X_reshaped)
            ytest_predicted_unscaled = unscale(ytest_predicted,value_min[0],value_max[0])
            ytest_predicted_unscaled_list.append(ytest_predicted_unscaled)
            test_y_unscaled = unscale(test_y, value_min[0], value_max[0])
            test_y_unscaled_list.append(test_y_unscaled)
            forecast321 = ytest_predicted_unscaled.reshape(-1,1)
            #forecast321 = test_y_unscaled_list
            forecast321 = pd.DataFrame(forecast321)
            forecast321.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/TestsetCSVs/24Lags/testsetcsv"+str(j)+".csv")
            #Predict on the valset
            yval_predicted = model.predict(val_X_reshaped)
            yval_predicted_unscaled = unscale(yval_predicted,value_min[0],value_max[0])
            yval_predicted_unscaled_list.append(yval_predicted_unscaled)
            val_y_unscaled = unscale(val_y, value_min[0], value_max[0])
            val_y_unscaled_list.append(val_y_unscaled)
            forecast322 = yval_predicted_unscaled.reshape(-1,1)
            #forecast322 = val_y_unscaled_list
            forecast322 = pd.DataFrame(forecast322)
            forecast322.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/ValsetCSVs/24Lags/valsetcsv"+str(j)+".csv")
            m =[]
            m1=[]
            m2=[]
            m3=[]
            m4=[]
            m5=[]
            stdpredtrain = []
            stdpredtest  = []
            stdvalpred   = []
            valy = np.asarray(val_y_unscaled_list)
            valy = valy.reshape(-1,1)
            q=np.asarray(train_y_unscaled_list).reshape(-1,1)
            k=np.asarray(test_y_unscaled_list).reshape(-1,1) 
            trainy=q.reshape(-1,1)
            testy=k.reshape(-1,1)
            for i in range(len(q)):
              if trainy[i]>0 or trainy[i]<0:
                a  = (np.asarray(train_y_unscaled_list).reshape(-1,1)[i]-np.asarray(ytrain_predicted_unscaled_list).reshape(-1,1)[i])**2
                a1 = abs((np.asarray(train_y_unscaled_list).reshape(-1,1)[i]-np.asarray(ytrain_predicted_unscaled_list).reshape(-1,1)[i])/np.asarray(train_y_unscaled_list).reshape(-1,1)[i])
                b1 = (np.asarray(ytrain_predicted_unscaled_list).reshape(-1,1)[i]-np.mean(np.asarray(ytrain_predicted_unscaled_list).reshape(-1,1)))**2
                m.append(a)
                m1.append(a1)
                stdpredtrain.append(b1)
            for i in range(len(k)):
              if testy[i]>0 or testy[i]<0:
                a2 = ((np.asarray(test_y_unscaled_list)).reshape(-1,1)[i]-(np.asarray(ytest_predicted_unscaled_list)).reshape(-1,1)[i])**2
                a3 = abs(((np.asarray(test_y_unscaled_list)).reshape(-1,1)[i]-(np.asarray(ytest_predicted_unscaled_list)).reshape(-1,1)[i])/(np.asarray(test_y_unscaled_list)).reshape(-1,1)[i])
                b2 = (np.asarray(ytest_predicted_unscaled_list).reshape(-1,1)[i]-np.mean(np.asarray(ytest_predicted_unscaled_list).reshape(-1,1)))**2
                m2.append(a2)
                m3.append(a3)
                stdpredtest.append(b2)
                tg = np.asarray(yval_predicted_unscaled_list).reshape(-1,1)
            for i in range(len(tg)):
                if valy[i]>0 or valy[i]<0:
                    a4 = (np.asarray(val_y_unscaled_list).reshape(-1,1)[i]-np.asarray(np.asarray(yval_predicted_unscaled_list)).reshape(-1,1)[i])**2
                    b3 = ((np.asarray(yval_predicted_unscaled_list)).reshape(-1,1)[i]-np.mean(np.asarray(yval_predicted_unscaled_list)).reshape(-1,1))**2
                    a5 = abs((np.asarray(val_y_unscaled_list).reshape(-1,1)[i]-np.asarray(yval_predicted_unscaled_list).reshape(-1,1)[i])/np.asarray(val_y_unscaled_list).reshape(-1,1)[i])
                    m4.append(a4)
                    m5.append(a5)
                    stdvalpred.append(b3)
                C1=np.sqrt(np.mean(m))
                V1=np.mean(m1)*100
                C2=np.sqrt(np.mean(m2))
                V2=np.mean(m3)*100
                C3=np.sqrt(np.mean(m4))
                V3=np.mean(m5)*100
                B1=np.sqrt(np.mean(stdpredtrain))
                B2=np.sqrt(np.mean(stdpredtest))
                B3=np.sqrt(np.mean(stdvalpred))
                
            #print('TS_RMSE:',C1,'TS_MAPE:',V1,'CS_RMSE:',C2,'CS_MAPE:',V2,'Test_RMSE:',C3,'Test_MAPE:',V3)            
            #trainNSE_calc = NSE(train_y_unscaled,ytrain_predicted_unscaled)
            #trainRMSE_calc = RMSE(train_y_unscaled,ytrain_predicted_unscaled)
            #trainMAPE_calc = MAPE(train_y_unscaled,ytrain_predicted_unscaled)
            #trainNSE_calc_list.append(trainNSE_calc)
            trainRMSE_calc_list.append(C1)
            trainMAPE_calc_list.append(V1)
            trainstd_calc_list.append(B1)
            #testNSE_calc = NSE(test_y_unscaled,ytest_predicted_unscaled)
            #testRMSE_calc = RMSE(test_y_unscaled,ytest_predicted_unscaled)
            #testMAPE_calc = MAPE(test_y_unscaled,ytest_predicted_unscaled)
            #testNSE_calc_list.append(testNSE_calc)
            testRMSE_calc_list.append(C2)
            testMAPE_calc_list.append(V2)
            teststd_calc_list.append(B2)
            #valNSE_calc = NSE(val_y_unscaled,yval_predicted_unscaled)
            #valRMSE_calc = RMSE(val_y_unscaled,yval_predicted_unscaled)
            #valMAPE_calc = MAPE(val_y_unscaled,yval_predicted_unscaled)
            #valNSE_calc_list.append(valNSE_calc)
            valRMSE_calc_list.append(C3)
            valMAPE_calc_list.append(V3)
            valstd_calc_list.append(B3)
        return trainstd_calc_list,teststd_calc_list,valstd_calc_list, trainRMSE_calc_list,trainMAPE_calc_list,testRMSE_calc_list,testMAPE_calc_list,valRMSE_calc_list,valMAPE_calc_list,train_y_unscaled_list,ytrain_predicted_unscaled_list,test_y_unscaled_list,ytest_predicted_unscaled_list,val_y_unscaled_list,yval_predicted_unscaled_list


trainstd_calc_list,teststd_calc_list,valstd_calc_list,trainRMSE_calc_list,trainMAPE_calc_list,testRMSE_calc_list,testMAPE_calc_list,valRMSE_calc_list,valMAPE_calc_list,train_y_unscaled_list,ytrain_predicted_unscaled_list,test_y_unscaled_list,ytest_predicted_unscaled_list,val_y_unscaled_list,yval_predicted_unscaled_list = features_prediction(n_obs,train,test)

#Save performances in the drive
pd.DataFrame(trainstd_calc_list).to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/stdtrainset.csv")
pd.DataFrame(teststd_calc_list).to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/stdtestset.csv")
pd.DataFrame(valstd_calc_list).to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/stdvalset.csv")
pd.DataFrame(trainRMSE_calc_list).to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/rmsetrainset.csv")
pd.DataFrame(trainMAPE_calc_list).to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/mapetrainset.csv")
pd.DataFrame(testRMSE_calc_list).to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/rmsetestset.csv")
pd.DataFrame(testMAPE_calc_list).to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/mapetestset.csv")
pd.DataFrame(valRMSE_calc_list).to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/rmsevalset.csv")
pd.DataFrame(valMAPE_calc_list).to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/mapevalset.csv")


##SAVE THE RMSE AND MAPE IN CSV FILE AND COMPUTE AVERAGE PERFORMANCES

In [0]:
import pandas as pd
import numpy as np
df  = pd.read_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/rmsetrainset.csv",index_col=0,header = 0)
dfm = pd.DataFrame(df).mean()
dfs = pd.DataFrame(df).std()
s1  = pd.DataFrame(dfm).rename({'0':'trainset_mrmse'})
s2  = pd.DataFrame(dfs).rename({'0':'trainset_stdmrmse'})
RFreport = pd.concat([s1,s2],axis=1,ignore_index=False)
df1 = pd.read_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/mapetrainset.csv",index_col=0,header = 0)
dfm1= pd.DataFrame(df1).mean()
dfs1= pd.DataFrame(df1).std()
s11 = pd.DataFrame(dfm1).rename({'0':'trainset_mape'})
s21 = pd.DataFrame(dfs1).rename({'0':'trainset_stdmape'})
RFreport = pd.concat([RFreport,s11,s21],axis=1,ignore_index=False)
df2  = pd.read_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/rmsetestset.csv",index_col=0,header = 0)
dfm2 = pd.DataFrame(df2).mean()
dfs2 = pd.DataFrame(df2).std()
s12  = pd.DataFrame(dfm2).rename({'0':'testset_mrmse'})
s22  = pd.DataFrame(dfs2).rename({'0':'testset_stdmrmse'})
RFreport = pd.concat([RFreport,s12,s22],axis=1,ignore_index=False)
df3  = pd.read_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/mapetestset.csv",index_col=0,header = 0)
dfm3 = pd.DataFrame(df3).mean()
dfs3 = pd.DataFrame(df3).std()
s13  = pd.DataFrame(dfm3).rename({'0':'testset_mape'})
s23  = pd.DataFrame(dfs3).rename({'0':'testset_stdmape'})
RFreport = pd.concat([RFreport,s13,s23],axis=1,ignore_index=False)
df4  = pd.read_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/rmsevalset.csv",index_col=0,header = 0)
dfm4 = pd.DataFrame(df4).mean()
dfs4 = pd.DataFrame(df4).std()
s14  = pd.DataFrame(dfm4).rename({'0':'valset_rmse'})
s24  = pd.DataFrame(dfs4).rename({'0':'valset_stdrmse'})
RFreport = pd.concat([RFreport,s14,s24],axis=1,ignore_index=False)
df5  = pd.read_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/mapevalset.csv",index_col=0,header = 0)
dfm5 = pd.DataFrame(df5).mean()
dfs5 = pd.DataFrame(df5).std()
s15  = pd.DataFrame(dfm5).rename({'0':'valset_mape'})
s25  = pd.DataFrame(dfs5).rename({'0':'valset_stdmape'})
RFreport = pd.concat([RFreport,s15,s25],axis=1,ignore_index=False)
RFreport = RFreport.transpose()
RFreport = pd.DataFrame(RFreport.sum(axis=0))

RFreport.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Performances/24Lags/Lag24_Sto_performancereport.csv")                 

##CONCATENATE RESULTS OBTAINED FROM 30 SIMULATIONS IN ONE CSV FILE AND COMPUTE AVERAGE

In [0]:
#Concatenate the csvfiles and compute the mean
import pandas as pd
import glob
#Concatenate train, val and testsets and compute the mean values that are used as final output

path = r'./gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/TrainsetCSVs/24Lags' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

frame = pd.concat(li, axis=1, ignore_index=False)
frame1 = frame.mean(axis = 1) 
frame.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/TrainsetCSVs/24Lags/concat_predictions.csv")
frame1.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/TrainsetCSVs/24Lags/mean_predictions.csv")

path = r'./gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/TestsetCSVs/24Lags' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

frame = pd.concat(li, axis=1, ignore_index=False)
frame1 = frame.mean(axis = 1) 
frame.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/TestsetCSVs/24Lags/concat_predictions.csv")
frame1.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/TestsetCSVs/24Lags/mean_predictions.csv")

path = r'./gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/ValsetCSVs/24Lags' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=0, header=0)
    li.append(df)

frame = pd.concat(li, axis=1, ignore_index=False)
#rame1 = frame.mean(axis = 1) 
frame.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/ValsetCSVs/24Lags/concat_predictions.csv")
frame1.to_csv("/content/gdrive/My Drive/PHD_THESIS/Model_for_WDDE/SCM/LULSGR_Forecasting/Nigeria/Livestocks_GRF/Sheep/Results/ValsetCSVs/24Lags/mean_predictions.csv")

##CONCATENATE LOSS HISTORY IN ONE CSV AND COMPUTE AVERAGE

In [0]:
import pandas as pd
import glob
path = r'./content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Loss_history/24Lags' # use your path
for j in range(30):
  all_files = glob.glob(path + "/Lag24_history"+str(j)+".csv")

li = []
li1 = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=0, header=0)
    df1 = df['loss']
    df2 = df['val_loss']
    li.append(df1)
    li1.append(df2)

frame1 = pd.concat(li, axis=1, ignore_index=False)
frame2 = pd.concat(li1, axis=1, ignore_index=False)
frame3 = frame1.mean(axis = 1) 
frame4 = frame2.mean(axis = 1)
frame1.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Loss_history/24Lags/concat_loss_function.csv")
frame2.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Loss_history/24Lags/concat_val_loss_function.csv")
frame3.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Loss_history/24Lags/mean_loss_function.csv")
frame4.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Loss_history/24Lags/mean_val_loss_function.csv")
loss = pd.concat([frame3,frame4],axis = 1,ignore_index=False)
loss.to_csv("/content/gdrive/My Drive/PHD_THESIS/PAPER1/STORAGE_FORECAST/UVLSTM/Results/Loss_history/24Lags/mean_loss_val_loss_functions.csv")